## 本地使用Visual Studio Code开发

### 环境要求
安装Jupyter的插件
安装Anaconda

# 依赖环境安装（powershell/bash）
conda activate faiss-env 
uv pip install torch torchvision
uv pip install pillow opencv-python

# CNN图像特征提取

ResNet通过残差连接解决深层网络梯度消失问题，是提取图像特征的经典模型。我们使用预训练的ResNet50模型，去除分类层后获取图像特征向量。

In [12]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet50_Weights
from PIL import Image
import numpy as np
from pathlib import Path

# 1. 设备配置（优先使用GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 加载预训练ResNet模型并修改为特征提取器（修复兼容性问题）
weights = ResNet50_Weights.IMAGENET1K_V2
model = models.resnet50(weights=weights).to(device)
# 去除最后两层（全局平均池化层后直接输出特征，无需分类层）
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
feature_extractor.eval()  # 进入评估模式，禁用Dropout等

# 3. 图像预处理（与预训练模型要求一致，修复缩放问题）
transform = transforms.Compose([
    transforms.Resize(256),  # 先缩放到256（短边），保持比例
    transforms.CenterCrop(224),  # 中心裁剪到224×224（ResNet标准输入）
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet均值
                         std=[0.229, 0.224, 0.225])   # ImageNet标准差
])

# 4. 定义特征提取函数（完善类型注解、边界条件）
def extract_image_feature(image_path: str | Path) -> np.ndarray | None:
    """提取单张图像的特征向量"""
    try:
        image = Image.open(str(image_path)).convert("RGB")  # 支持Path对象，转为RGB格式
        input_tensor = transform(image).unsqueeze(0).to(device)  # 增加批次维度
        
        # 无梯度计算加速
        with torch.no_grad():
            feature = feature_extractor(input_tensor)
        
        # 特征向量处理（展平为1D向量并归一化）
        feature_vector = feature.squeeze().cpu().numpy()
        # L2归一化（增加边界条件，避免除以0）
        norm = np.linalg.norm(feature_vector)
        feature_vector = feature_vector / norm if norm > 1e-6 else feature_vector
        return feature_vector.astype(np.float32)
    except Exception as e:
        print(f"特征提取失败：{image_path} -> {e}")
        return None

# 5. 测试特征提取（优化路径处理）
image_dir = Path("D:\Project\easy-vectordb-practice\docs\images")  # 去掉./，规范绝对路径
image_paths = [p for p in image_dir.iterdir() if p.is_file()]

# 提取文件夹内所有图像的特征
image_features = []
image_metadata = []
for path in image_paths[:5]:  # 先处理5张图像测试
    vec = extract_image_feature(path)
    if vec is not None:
        image_features.append(vec)
        # 构建元数据（图像路径、论坛图片ID等，实际中可从文件名解析）
        image_metadata.append({
            "image_path": str(path),
            "product_id": path.stem,  # Path对象直接调用stem，更简洁
            "category": "sample"  # 可根据实际分类修改
        })

if image_features:  # 增加判断，避免空数组报错
    image_features = np.array(image_features)
    print(f"提取特征向量维度：{image_features.shape[1]}")  # 输出：2048
    print(f"成功提取特征的图像数量：{image_features.shape[0]}")
else:
    print("未提取到任何特征向量")

提取特征向量维度：2048
成功提取特征的图像数量：5


# 图像检索库构建与检索
图像特征向量维度通常为2048维（ResNet系列），需结合数据规模选择索引类型。此处采用IndexIVFFlat实现高效近似搜索。

In [13]:
import faiss
import json
import numpy as np
from pathlib import Path

# 假设以下变量已在之前的代码中定义（需确保存在）
# image_features: np.ndarray (n, d)，float32类型
# image_metadata: list，存储图片元数据
# extract_image_feature: 特征提取函数

# 1. 配置路径
db_dir = Path("./image_search_db")
db_dir.mkdir(parents=True, exist_ok=True)
index_path = db_dir / "image_index.index"
metadata_path = db_dir / "image_metadata.json"

# 2. 边界条件：检查特征向量是否为空
if len(image_features) == 0:
    print("错误：没有可处理的特征向量，终止索引构建")
else:
    # 确保特征向量为float32类型（FAISS的硬性要求）
    image_features = np.array(image_features).astype(np.float32)
    d = image_features.shape[1]  # 特征维度（如2048）
    n = len(image_features)     # 特征数量

    # 3. 动态计算nlist（经验值：数据量的平方根，且不小于1、不大于数据量）
    nlist = int(np.sqrt(n)) if n > 0 else 1
    nlist = max(nlist, 1)  # 至少1个聚类
    nlist = min(nlist, n)  # 不超过数据量（避免聚类数大于数据量）

    # 4. 构建FAISS索引（根据数据量选择索引类型）
    if n < 10000:  # 小规模数据：使用精确检索的IndexFlatL2（无需训练）
        print("小规模数据，使用IndexFlatL2精确检索")
        index = faiss.IndexFlatL2(d)
    else:  # 大规模数据：使用IndexIVFFlat近似检索
        print("大规模数据，使用IndexIVFFlat近似检索")
        quantizer = faiss.IndexFlatL2(d)  # 量化器（基于L2距离）
        index = faiss.IndexIVFFlat(quantizer, d, nlist)

    # 5. 训练索引（仅IVF类索引需要训练）
    if isinstance(index, faiss.IndexIVFFlat) and not index.is_trained:
        index.train(image_features)
        print("索引训练完成")

    # 6. 添加向量并保存
    index.add(image_features)
    faiss.write_index(index, str(index_path))
    with open(metadata_path, "w", encoding="utf-8") as f:
        json.dump(image_metadata, f, ensure_ascii=False, indent=2)
    print(f"图像检索库构建完成，包含{index.ntotal}个特征向量")

    # 7. 加载索引与元数据
    loaded_index = faiss.read_index(str(index_path))
    with open(metadata_path, "r", encoding="utf-8") as f:
        loaded_img_metadata = json.load(f)

    # 8. 相似图像检索测试
    test_image_path = "./test_image.png"  # 测试查询图像
    test_vec = extract_image_feature(test_image_path)

    # 边界条件：检查测试向量是否提取成功
    if test_vec is not None:
        test_vec = test_vec.reshape(1, -1).astype(np.float32)  # 转为float32并增加batch维度

        # 调整检索精度（仅IVF类索引有效，nprobe越大精度越高，速度越慢）
        if isinstance(loaded_index, faiss.IndexIVFFlat):
            loaded_index.nprobe = 3  # 经验值，可根据需求调整

        k = 3  # 返回Top-3相似图像
        distances, indices = loaded_index.search(test_vec, k)

        # 解析结果（L2距离越小相似度越高）
        print("\n相似图像检索结果：")
        for i in range(k):
            idx = indices[0][i]
            # 防止索引越界（比如数据量不足k个）
            if idx < 0 or idx >= len(loaded_img_metadata):
                print(f"排名{i+1}：无匹配结果")
                continue
            print(f"排名{i+1}：L2距离{distances[0][i]:.4f}")
            print(f"论坛图片ID：{loaded_img_metadata[idx]['product_id']}")
            print(f"图像路径：{loaded_img_metadata[idx]['image_path']}\n")
    else:
        print(f"错误：测试图片{test_image_path}特征提取失败")

小规模数据，使用IndexFlatL2精确检索
图像检索库构建完成，包含5个特征向量

相似图像检索结果：
排名1：L2距离0.0000
论坛图片ID：1a0e47d1-a295-42a9-a3c8-3f44d97f26f5
图像路径：D:\Project\easy-vectordb-practice\docs\images\1a0e47d1-a295-42a9-a3c8-3f44d97f26f5.png

排名2：L2距离0.2255
论坛图片ID：75ceb54f-534c-4e3e-b829-796c2e678383
图像路径：D:\Project\easy-vectordb-practice\docs\images\75ceb54f-534c-4e3e-b829-796c2e678383.png

排名3：L2距离0.2547
论坛图片ID：7f3f87c1-07fc-43eb-8463-970ea69dd458
图像路径：D:\Project\easy-vectordb-practice\docs\images\7f3f87c1-07fc-43eb-8463-970ea69dd458.png



# Streamlist 访问

conda activate faiss-env
uv pip install -r requirements.txt

cd d:\Project\easy-vectordb-practice\faiss-pic-demo
streamlit run streamlit_image_search.py